In [17]:
# Import Packages
import numpy as np
import scipy.stats as stats
import os
import pickle
import scipy.io as scio
from ncdump_python3 import ncdump
import pickle
import Jacks_Functions as JF
import Area_Avg
import matplotlib
from matplotlib import pyplot as plt
from scipy.interpolate import griddata
from netCDF4 import Dataset
import warnings
warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True)

In [18]:
#Load in Dimensions
LLL = Dataset('LatLon.nc')
WACCM4_Lat = np.squeeze(LLL.variables['lat'])
WACCM4_Lon = np.squeeze(LLL.variables['lon'])
LLL.close()

Source_File = Dataset('Kernels/Interpolated/CAM3_Kernels.nc')

CAM3_Lat = np.squeeze(Source_File.variables['lat'])
CAM3_Lon = np.squeeze(Source_File.variables['lon'])

cx,cy = np.meshgrid(CAM3_Lon,CAM3_Lat)
wx,wy = np.meshgrid(WACCM4_Lon,WACCM4_Lat)

RCP45_PS = pickle.load(open('RCP45_Base_PS.pickle','rb'))
#interpolate the surface pressure to the CAM3 grid
RCP45_PS_interp = np.zeros([12,64,128])
for i in range(12):
    RCP45_PS_interp[i,:,:] = griddata(\
    (wx.flatten(),wy.flatten()),RCP45_PS[i,:,:].flatten(),\
    (cx,cy),method='linear')/100 #swap to hpa in the process
    
#Lists for iterating
Months = np.arange(0,12)
FP_Ensemble_Members = np.arange(5)

In [19]:
#Tile the plevs to match the shape of the kernel
CMIP_plevs_scalar = np.squeeze(Source_File.variables['lev'])

A = np.tile(CMIP_plevs_scalar.T,(CAM3_Lon.size,1))
B = np.transpose(A[:,:,None],(0,1,2))
C = np.tile(B,(1,1,CAM3_Lat.size))
D = np.transpose(C[:,:,:,None],(0,1,2,3))
E = np.tile(D,(1,1,1,12))
CMIP_plevs = np.swapaxes(E,3,0)
CMIP_plevs.shape

(12, 17, 64, 128)

In [20]:
#Calculate pressure level thickness using CMIP pressure levels and surface pressure
dp = np.zeros([12,17,64,128])

for i in range(12):
    for j in range(64):
        for k in range(128):
            dp[i,:,j,k] = JF.PlevThck(\
            PS = RCP45_PS_interp[i,j,k],plevs=CMIP_plevs_scalar,\
            p_top = min(CMIP_plevs_scalar))
            
dp = np.ma.masked_invalid(dp[:,:,:,:]/100) #CAM3 Kernel units are per 100 hPa

In [21]:
#climate change responses
RCP45_dST_CAM3Grid = pickle.load(open(\
   "Future Projection CC Responses/RCP45_dST_CAM3Grid.pickle","rb"),encoding='latin1')
RCP45_dST_CAM3Grid = np.swapaxes(RCP45_dST_CAM3Grid,2,3)
RCP45_dAT_CAM3Grid = pickle.load(open(\
    "Future Projection CC Responses/RCP45_dAT_CAM3Grid.pickle","rb"),encoding='latin1')
RCP45_dAT_CAM3Grid = np.swapaxes(RCP45_dAT_CAM3Grid,3,4)

In [22]:
#estimation of the tropopause. 300hPA at the poles, raising
    #linearly to 100hpa at the tropics

p_tropopause_zonalmean_linear = np.zeros([64])

for y in range(64):
    if y <= len(CAM3_Lat)/2:
        if y == 0:
            p_tropopause_zonalmean_linear[y] = 300
        elif y == len(CAM3_Lat)/2:
            p_tropopause_zonalmean_linear[y] = 100
        else:
            p_tropopause_zonalmean_linear[y] = \
            p_tropopause_zonalmean_linear[y-1]\
            -(200/(len(CAM3_Lat)/2-1))
    else:
        if y == len(CAM3_Lat)/2:
            p_tropopause_zonalmean_linear[y] = 100
        elif y == len(CAM3_Lat):
            p_tropopause_zonalmean_linear[y] = 300
        else:
            p_tropopause_zonalmean_linear[y] = \
            p_tropopause_zonalmean_linear[y-1]\
            +(200/(len(CAM3_Lat)/2-1))
            
Q = np.tile(p_tropopause_zonalmean_linear.T,(CAM3_Lon.size,1))
R = np.transpose(Q[:,:,None],(0,1,2))
S = np.tile(R,(1,1,17))
T = np.transpose(S[:,:,:,None],(0,1,2,3))
U = np.tile(T,(1,1,1,12))
V = np.swapaxes(U,3,0)
p_tropopause_CAM3 = np.swapaxes(V,2,1)

#how does it look?
p_tropopause_CAM3.shape

(12, 17, 64, 128)

In [23]:
#CAM3
Kernel_CAM3_TA = np.ma.masked_greater(\
    np.squeeze(Source_File.variables['Ta_TOA']),1e10)
Kernel_CAM3_TA_clear = np.ma.masked_greater(\
    np.squeeze(Source_File.variables['Ta_TOA_CLR']),1e10)

Kernel_CAM3_TS = np.ma.masked_greater(\
    np.squeeze(Source_File.variables['Ts_TOA']),1e10)
Kernel_CAM3_TS_clear = np.ma.masked_greater(\
    np.squeeze(Source_File.variables['Ts_TOA_CLR']),1e10)

In [24]:
#arrays for total temperature feedback
CAM3_RCP45_dLW_ts = np.zeros([FP_Ensemble_Members.size,12,64,128])
CAM3_RCP45_dLW_ts_clear = np.zeros([FP_Ensemble_Members.size,12,64,128])
CAM3_RCP45_dLW_ta = np.zeros([FP_Ensemble_Members.size,12,64,128])
CAM3_RCP45_dLW_ta_clear = np.zeros([FP_Ensemble_Members.size,12,64,128])
CAM3_RCP45_dta_tropo = np.zeros([FP_Ensemble_Members.size,12,17,64,128])
CAM3_RCP45_dLW_t = np.zeros([FP_Ensemble_Members.size,12,64,128])
CAM3_RCP45_dLW_t_clear = np.zeros([FP_Ensemble_Members.size,12,64,128])

#arrays for planck feedback
CAM3_RCP45_dts3d = np.zeros([FP_Ensemble_Members.size,12,17,64,128])
CAM3_RCP45_dts3d_tropo = np.zeros([FP_Ensemble_Members.size,12,17,64,128])
CAM3_RCP45_dLW_uniform = np.zeros([FP_Ensemble_Members.size,12,64,128])
CAM3_RCP45_dLW_planck = np.zeros([FP_Ensemble_Members.size,12,64,128])

#arrays for Lapse Rate feedback
CAM3_RCP45_dt_lapserate = np.zeros([FP_Ensemble_Members.size,12,17,64,128])
CAM3_RCP45_dLW_lapserate = np.zeros([FP_Ensemble_Members.size,12,64,128])
CAM3_RCP45_dLW_lapserate_clear = np.zeros([FP_Ensemble_Members.size,12,64,128])


for i in range(FP_Ensemble_Members.size):
    
    #Multiply surface temperature change by surface temp kernel
    CAM3_RCP45_dLW_ts[i,:,:,:] = Kernel_CAM3_TS*RCP45_dST_CAM3Grid[i,:,:,:]
    CAM3_RCP45_dLW_ts_clear[i,:,:,:] = Kernel_CAM3_TS_clear*RCP45_dST_CAM3Grid[i,:,:,:]
    
    #Mask out stratosphere air temp change values by changing them to zero
    CAM3_RCP45_dta_tropo[i,:,:,:,:] = \
    RCP45_dAT_CAM3Grid[i,:,:,:,:]*(CMIP_plevs>=p_tropopause_CAM3)
    
    #multiply air temp change and air temp kernel and dp, then integrate
        #up to the tropopause
    CAM3_RCP45_dLW_ta[i,:,:,:] = np.ma.sum(\
    Kernel_CAM3_TA*CAM3_RCP45_dta_tropo[i,:,:,:,:]*dp,1)
    
    CAM3_RCP45_dLW_ta_clear[i,:,:,:] = np.ma.sum(\
    Kernel_CAM3_TA_clear*CAM3_RCP45_dta_tropo[i,:,:,:,:]*dp,1)
    
    # add the surface and air temperature feedback responses
    CAM3_RCP45_dLW_t[i,:,:,:] = CAM3_RCP45_dLW_ta[i,:,:,:]+CAM3_RCP45_dLW_ts[i,:,:,:]
    
    CAM3_RCP45_dLW_t_clear[i,:,:,:] = \
    CAM3_RCP45_dLW_ta_clear[i,:,:,:]+CAM3_RCP45_dLW_ts_clear[i,:,:,:]
    
    #moving on to planck feedback
    #Project surface temperature throughout the atmosphere
    CAM3_RCP45_dts3d[i,:,:,:,:] = np.swapaxes(np.swapaxes(np.tile(np.transpose(\
                        RCP45_dST_CAM3Grid[i,:,:,:][:,:,:,None],(0,1,2,3))\
                        ,(1,1,1,17))\
                        ,3,1)\
                        ,2,3)
    
    #mask stratosphere for new uniform temp change array
    CAM3_RCP45_dts3d_tropo[i,:,:,:,:] = \
    CAM3_RCP45_dts3d[i,:,:,:,:]*(CMIP_plevs>=p_tropopause_CAM3)
    
    #multiply uniform temp change with air temp kernel, then integrate
        #up to the tropopause
    CAM3_RCP45_dLW_uniform[i,:,:,:] = np.ma.sum(\
    Kernel_CAM3_TA*CAM3_RCP45_dts3d_tropo[i,:,:,:,:]*dp,1)
    
    # add the surface and air uniform temperature feedback responses
    CAM3_RCP45_dLW_planck[i,:,:,:] = \
    CAM3_RCP45_dLW_uniform[i,:,:,:]+CAM3_RCP45_dLW_ts[i,:,:,:]
    
    #Moving on to lapse rate
    #departure of temperature change from the surface
    CAM3_RCP45_dt_lapserate[i,:,:,:,:] = \
    CAM3_RCP45_dta_tropo[i,:,:,:,:]-CAM3_RCP45_dts3d_tropo[i,:,:,:,:]
    
    #multiply temperature change departure with air temp kernel, then integrate
        #up to the tropopause
    CAM3_RCP45_dLW_lapserate[i,:,:,:] = np.ma.sum(\
    Kernel_CAM3_TA*CAM3_RCP45_dt_lapserate[i,:,:,:,:]*dp,1)
    
    CAM3_RCP45_dLW_lapserate_clear[i,:,:,:] = np.ma.sum(\
    Kernel_CAM3_TA_clear*CAM3_RCP45_dt_lapserate[i,:,:,:,:]*dp,1)

In [25]:
CAM3_RCP45_LR_EnergyB_AA = np.zeros([FP_Ensemble_Members.size,12])
CAM3_RCP45_LR_EnergyB_GA = np.zeros([FP_Ensemble_Members.size,12])
CAM3_RCP45_PL_EnergyB_AA = np.zeros([FP_Ensemble_Members.size,12])
CAM3_RCP45_PL_EnergyB_GA = np.zeros([FP_Ensemble_Members.size,12])

for i in range(FP_Ensemble_Members.size):
    CAM3_RCP45_LR_EnergyB_AA[i,:] = Area_Avg.LatLonavg_Time(\
    CAM3_RCP45_dLW_lapserate[i,:,54:,:],CAM3_Lat[54:],CAM3_Lon)
    
    CAM3_RCP45_LR_EnergyB_GA[i,:] = Area_Avg.LatLonavg_Time(\
    CAM3_RCP45_dLW_lapserate[i,:,:,:],CAM3_Lat,CAM3_Lon)
    
    CAM3_RCP45_PL_EnergyB_AA[i,:] = Area_Avg.LatLonavg_Time(\
    CAM3_RCP45_dLW_planck[i,:,54:,:],CAM3_Lat[54:],CAM3_Lon)
    
    CAM3_RCP45_PL_EnergyB_GA[i,:] = Area_Avg.LatLonavg_Time(\
    CAM3_RCP45_dLW_planck[i,:,:,:],CAM3_Lat,CAM3_Lon)

In [26]:
CAM3_RCP45_Planck_ZM = np.mean(CAM3_RCP45_dLW_planck,axis=(1,3))
CAM3_RCP45_Planck_ZM_file = open('CAM3_RCP45_Planck_ZM.pickle','wb')
pickle.dump(CAM3_RCP45_Planck_ZM,CAM3_RCP45_Planck_ZM_file)
CAM3_RCP45_Planck_ZM_file.close()

In [10]:
RCP45_dST_CAM3Grid_AA = np.zeros([5,12])
RCP45_dST_CAM3Grid_GA = np.zeros([5,12])

for i in range(FP_Ensemble_Members.size):
    RCP45_dST_CAM3Grid_AA[i,:] = Area_Avg.LatLonavg_Time(\
    RCP45_dST_CAM3Grid[i,:,54:,:],CAM3_Lat[54:],CAM3_Lon)
    
    RCP45_dST_CAM3Grid_GA[i,:] = Area_Avg.LatLonavg_Time(\
    RCP45_dST_CAM3Grid[i,:,:,:],CAM3_Lat,CAM3_Lon)

In [11]:
CAM3_RCP45_LR_Feedback_AA = CAM3_RCP45_LR_EnergyB_AA/RCP45_dST_CAM3Grid_AA
CAM3_RCP45_LR_Feedback_GA = CAM3_RCP45_LR_EnergyB_GA/RCP45_dST_CAM3Grid_GA

CAM3_RCP45_PL_Feedback_AA = CAM3_RCP45_PL_EnergyB_AA/RCP45_dST_CAM3Grid_AA
CAM3_RCP45_PL_Feedback_GA = CAM3_RCP45_PL_EnergyB_GA/RCP45_dST_CAM3Grid_GA

In [38]:
CAM3_RCP45_LR_EnergyB_Annual_AA = np.mean(CAM3_RCP45_LR_EnergyB_AA,axis=1)
CAM3_RCP45_LR_EnergyB_SON_AA = np.mean(CAM3_RCP45_LR_EnergyB_AA[:,8:11],axis=1)
CAM3_RCP45_LR_EnergyB_JJA_AA = np.mean(CAM3_RCP45_LR_EnergyB_AA[:,5:8],axis=1)
CAM3_RCP45_LR_EnergyB_MAM_AA = np.mean(CAM3_RCP45_LR_EnergyB_AA[:,2:5],axis=1)
CAM3_RCP45_LR_EnergyB_DJF_AA = np.mean((CAM3_RCP45_LR_EnergyB_AA[:,0],
                                       CAM3_RCP45_LR_EnergyB_AA[:,1],\
                                       CAM3_RCP45_LR_EnergyB_AA[:,11]),axis=0)

CAM3_RCP45_LR_EnergyB_Annual_GA = np.mean(CAM3_RCP45_LR_EnergyB_GA,axis=1)
CAM3_RCP45_LR_EnergyB_SON_GA = np.mean(CAM3_RCP45_LR_EnergyB_GA[:,8:11],axis=1)
CAM3_RCP45_LR_EnergyB_JJA_GA = np.mean(CAM3_RCP45_LR_EnergyB_GA[:,5:8],axis=1)
CAM3_RCP45_LR_EnergyB_MAM_GA = np.mean(CAM3_RCP45_LR_EnergyB_GA[:,2:5],axis=1)
CAM3_RCP45_LR_EnergyB_DJF_GA = np.mean((CAM3_RCP45_LR_EnergyB_GA[:,0],
                                       CAM3_RCP45_LR_EnergyB_GA[:,1],\
                                       CAM3_RCP45_LR_EnergyB_GA[:,11]),axis=0)

In [39]:
CAM3_RCP45_PL_EnergyB_Annual_AA = np.mean(CAM3_RCP45_PL_EnergyB_AA,axis=1)
CAM3_RCP45_PL_EnergyB_SON_AA = np.mean(CAM3_RCP45_PL_EnergyB_AA[:,8:11],axis=1)
CAM3_RCP45_PL_EnergyB_JJA_AA = np.mean(CAM3_RCP45_PL_EnergyB_AA[:,5:8],axis=1)
CAM3_RCP45_PL_EnergyB_MAM_AA = np.mean(CAM3_RCP45_PL_EnergyB_AA[:,2:5],axis=1)
CAM3_RCP45_PL_EnergyB_DJF_AA = np.mean((CAM3_RCP45_PL_EnergyB_AA[:,0],
                                       CAM3_RCP45_PL_EnergyB_AA[:,1],\
                                       CAM3_RCP45_PL_EnergyB_AA[:,11]),axis=0)

CAM3_RCP45_PL_EnergyB_Annual_GA = np.mean(CAM3_RCP45_PL_EnergyB_GA,axis=1)
CAM3_RCP45_PL_EnergyB_SON_GA = np.mean(CAM3_RCP45_PL_EnergyB_GA[:,8:11],axis=1)
CAM3_RCP45_PL_EnergyB_JJA_GA = np.mean(CAM3_RCP45_PL_EnergyB_GA[:,5:8],axis=1)
CAM3_RCP45_PL_EnergyB_MAM_GA = np.mean(CAM3_RCP45_PL_EnergyB_GA[:,2:5],axis=1)
CAM3_RCP45_PL_EnergyB_DJF_GA = np.mean((CAM3_RCP45_PL_EnergyB_GA[:,0],
                                       CAM3_RCP45_PL_EnergyB_GA[:,1],\
                                       CAM3_RCP45_PL_EnergyB_GA[:,11]),axis=0)

In [40]:
CAM3_RCP45_LR_Feedback_Annual_AA = np.mean(CAM3_RCP45_LR_Feedback_AA,axis=1)
CAM3_RCP45_LR_Feedback_SON_AA = np.mean(CAM3_RCP45_LR_Feedback_AA[:,8:11],axis=1)
CAM3_RCP45_LR_Feedback_JJA_AA = np.mean(CAM3_RCP45_LR_Feedback_AA[:,5:8],axis=1)
CAM3_RCP45_LR_Feedback_MAM_AA = np.mean(CAM3_RCP45_LR_Feedback_AA[:,2:5],axis=1)
CAM3_RCP45_LR_Feedback_DJF_AA = np.mean((CAM3_RCP45_LR_Feedback_AA[:,0],
                                       CAM3_RCP45_LR_Feedback_AA[:,1],\
                                       CAM3_RCP45_LR_Feedback_AA[:,11]),axis=0)

CAM3_RCP45_LR_Feedback_Annual_GA = np.mean(CAM3_RCP45_LR_Feedback_GA,axis=1)
CAM3_RCP45_LR_Feedback_SON_GA = np.mean(CAM3_RCP45_LR_Feedback_GA[:,8:11],axis=1)
CAM3_RCP45_LR_Feedback_JJA_GA = np.mean(CAM3_RCP45_LR_Feedback_GA[:,5:8],axis=1)
CAM3_RCP45_LR_Feedback_MAM_GA = np.mean(CAM3_RCP45_LR_Feedback_GA[:,2:5],axis=1)
CAM3_RCP45_LR_Feedback_DJF_GA = np.mean((CAM3_RCP45_LR_Feedback_GA[:,0],
                                       CAM3_RCP45_LR_Feedback_GA[:,1],\
                                       CAM3_RCP45_LR_Feedback_GA[:,11]),axis=0)

In [41]:
CAM3_RCP45_PL_Feedback_Annual_AA = np.mean(CAM3_RCP45_PL_Feedback_AA,axis=1)
CAM3_RCP45_PL_Feedback_SON_AA = np.mean(CAM3_RCP45_PL_Feedback_AA[:,8:11],axis=1)
CAM3_RCP45_PL_Feedback_JJA_AA = np.mean(CAM3_RCP45_PL_Feedback_AA[:,5:8],axis=1)
CAM3_RCP45_PL_Feedback_MAM_AA = np.mean(CAM3_RCP45_PL_Feedback_AA[:,2:5],axis=1)
CAM3_RCP45_PL_Feedback_DJF_AA = np.mean((CAM3_RCP45_PL_Feedback_AA[:,0],
                                       CAM3_RCP45_PL_Feedback_AA[:,1],\
                                       CAM3_RCP45_PL_Feedback_AA[:,11]),axis=0)

CAM3_RCP45_PL_Feedback_Annual_GA = np.mean(CAM3_RCP45_PL_Feedback_GA,axis=1)
CAM3_RCP45_PL_Feedback_SON_GA = np.mean(CAM3_RCP45_PL_Feedback_GA[:,8:11],axis=1)
CAM3_RCP45_PL_Feedback_JJA_GA = np.mean(CAM3_RCP45_PL_Feedback_GA[:,5:8],axis=1)
CAM3_RCP45_PL_Feedback_MAM_GA = np.mean(CAM3_RCP45_PL_Feedback_GA[:,2:5],axis=1)
CAM3_RCP45_PL_Feedback_DJF_GA = np.mean((CAM3_RCP45_PL_Feedback_GA[:,0],
                                       CAM3_RCP45_PL_Feedback_GA[:,1],\
                                       CAM3_RCP45_PL_Feedback_GA[:,11]),axis=0)

In [42]:
x = pickle.load(open('ENERGY BUDGET/Lapse Rate/RCP45/CAM3_RCP45_LR_EnergyB_AA.pickle','rb'))
print(CAM3_RCP45_LR_EnergyB_Annual_AA)
print(x)

[3.04162345 2.81235463 2.79489744 3.02007176 3.57145853]
[2.07971119 1.89473557 1.91254737 2.07129598 2.44430157]


In [43]:
CAM3_RCP45_LR_EnergyB_Annual_AA_file = open("CAM3_RCP45_LR_EnergyB_AA.pickle","wb")
pickle.dump(CAM3_RCP45_LR_EnergyB_Annual_AA,CAM3_RCP45_LR_EnergyB_Annual_AA_file)
CAM3_RCP45_LR_EnergyB_Annual_AA_file.close()

CAM3_RCP45_LR_EnergyB_SON_AA_file = open("CAM3_RCP45_LR_EnergyB_SON_AA.pickle","wb")
pickle.dump(CAM3_RCP45_LR_EnergyB_SON_AA,CAM3_RCP45_LR_EnergyB_SON_AA_file)
CAM3_RCP45_LR_EnergyB_SON_AA_file.close()

CAM3_RCP45_LR_EnergyB_JJA_AA_file = open("CAM3_RCP45_LR_EnergyB_JJA_AA.pickle","wb")
pickle.dump(CAM3_RCP45_LR_EnergyB_JJA_AA,CAM3_RCP45_LR_EnergyB_JJA_AA_file)
CAM3_RCP45_LR_EnergyB_JJA_AA_file.close()

CAM3_RCP45_LR_EnergyB_MAM_AA_file = open("CAM3_RCP45_LR_EnergyB_MAM_AA.pickle","wb")
pickle.dump(CAM3_RCP45_LR_EnergyB_MAM_AA,CAM3_RCP45_LR_EnergyB_MAM_AA_file)
CAM3_RCP45_LR_EnergyB_MAM_AA_file.close()

CAM3_RCP45_LR_EnergyB_DJF_AA_file = open("CAM3_RCP45_LR_EnergyB_DJF_AA.pickle","wb")
pickle.dump(CAM3_RCP45_LR_EnergyB_DJF_AA,CAM3_RCP45_LR_EnergyB_DJF_AA_file)
CAM3_RCP45_LR_EnergyB_DJF_AA_file.close()

CAM3_RCP45_LR_Feedback_Annual_AA_file = open("CAM3_RCP45_LR_Feedback_AA.pickle","wb")
pickle.dump(CAM3_RCP45_LR_Feedback_Annual_AA,CAM3_RCP45_LR_Feedback_Annual_AA_file)
CAM3_RCP45_LR_Feedback_Annual_AA_file.close()

CAM3_RCP45_LR_Feedback_SON_AA_file = open("CAM3_RCP45_LR_Feedback_SON_AA.pickle","wb")
pickle.dump(CAM3_RCP45_LR_Feedback_SON_AA,CAM3_RCP45_LR_Feedback_SON_AA_file)
CAM3_RCP45_LR_Feedback_SON_AA_file.close()

CAM3_RCP45_LR_Feedback_JJA_AA_file = open("CAM3_RCP45_LR_Feedback_JJA_AA.pickle","wb")
pickle.dump(CAM3_RCP45_LR_Feedback_JJA_AA,CAM3_RCP45_LR_Feedback_JJA_AA_file)
CAM3_RCP45_LR_Feedback_JJA_AA_file.close()

CAM3_RCP45_LR_Feedback_MAM_AA_file = open("CAM3_RCP45_LR_Feedback_MAM_AA.pickle","wb")
pickle.dump(CAM3_RCP45_LR_Feedback_MAM_AA,CAM3_RCP45_LR_Feedback_MAM_AA_file)
CAM3_RCP45_LR_Feedback_MAM_AA_file.close()

CAM3_RCP45_LR_Feedback_DJF_AA_file = open("CAM3_RCP45_LR_Feedback_DJF_AA.pickle","wb")
pickle.dump(CAM3_RCP45_LR_Feedback_DJF_AA,CAM3_RCP45_LR_Feedback_DJF_AA_file)
CAM3_RCP45_LR_Feedback_DJF_AA_file.close()

In [44]:
CAM3_RCP45_PL_EnergyB_Annual_AA_file = open("CAM3_RCP45_PL_EnergyB_AA.pickle","wb")
pickle.dump(CAM3_RCP45_PL_EnergyB_Annual_AA,CAM3_RCP45_PL_EnergyB_Annual_AA_file)
CAM3_RCP45_PL_EnergyB_Annual_AA_file.close()

CAM3_RCP45_PL_EnergyB_SON_AA_file = open("CAM3_RCP45_PL_EnergyB_SON_AA.pickle","wb")
pickle.dump(CAM3_RCP45_PL_EnergyB_SON_AA,CAM3_RCP45_PL_EnergyB_SON_AA_file)
CAM3_RCP45_PL_EnergyB_SON_AA_file.close()

CAM3_RCP45_PL_EnergyB_JJA_AA_file = open("CAM3_RCP45_PL_EnergyB_JJA_AA.pickle","wb")
pickle.dump(CAM3_RCP45_PL_EnergyB_JJA_AA,CAM3_RCP45_PL_EnergyB_JJA_AA_file)
CAM3_RCP45_PL_EnergyB_JJA_AA_file.close()

CAM3_RCP45_PL_EnergyB_MAM_AA_file = open("CAM3_RCP45_PL_EnergyB_MAM_AA.pickle","wb")
pickle.dump(CAM3_RCP45_PL_EnergyB_MAM_AA,CAM3_RCP45_PL_EnergyB_MAM_AA_file)
CAM3_RCP45_PL_EnergyB_MAM_AA_file.close()

CAM3_RCP45_PL_EnergyB_DJF_AA_file = open("CAM3_RCP45_PL_EnergyB_DJF_AA.pickle","wb")
pickle.dump(CAM3_RCP45_PL_EnergyB_DJF_AA,CAM3_RCP45_PL_EnergyB_DJF_AA_file)
CAM3_RCP45_PL_EnergyB_DJF_AA_file.close()

CAM3_RCP45_PL_Feedback_Annual_AA_file = open("CAM3_RCP45_PL_Feedback_AA.pickle","wb")
pickle.dump(CAM3_RCP45_PL_Feedback_Annual_AA,CAM3_RCP45_PL_Feedback_Annual_AA_file)
CAM3_RCP45_PL_Feedback_Annual_AA_file.close()

CAM3_RCP45_PL_Feedback_SON_AA_file = open("CAM3_RCP45_PL_Feedback_SON_AA.pickle","wb")
pickle.dump(CAM3_RCP45_PL_Feedback_SON_AA,CAM3_RCP45_PL_Feedback_SON_AA_file)
CAM3_RCP45_PL_Feedback_SON_AA_file.close()

CAM3_RCP45_PL_Feedback_JJA_AA_file = open("CAM3_RCP45_PL_Feedback_JJA_AA.pickle","wb")
pickle.dump(CAM3_RCP45_PL_Feedback_JJA_AA,CAM3_RCP45_PL_Feedback_JJA_AA_file)
CAM3_RCP45_PL_Feedback_JJA_AA_file.close()

CAM3_RCP45_PL_Feedback_MAM_AA_file = open("CAM3_RCP45_PL_Feedback_MAM_AA.pickle","wb")
pickle.dump(CAM3_RCP45_PL_Feedback_MAM_AA,CAM3_RCP45_PL_Feedback_MAM_AA_file)
CAM3_RCP45_PL_Feedback_MAM_AA_file.close()

CAM3_RCP45_PL_Feedback_DJF_AA_file = open("CAM3_RCP45_PL_Feedback_DJF_AA.pickle","wb")
pickle.dump(CAM3_RCP45_PL_Feedback_DJF_AA,CAM3_RCP45_PL_Feedback_DJF_AA_file)
CAM3_RCP45_PL_Feedback_DJF_AA_file.close()

In [45]:
CAM3_RCP45_LR_EnergyB_Annual_GA_file = open("CAM3_RCP45_LR_EnergyB_GA.pickle","wb")
pickle.dump(CAM3_RCP45_LR_EnergyB_Annual_GA,CAM3_RCP45_LR_EnergyB_Annual_GA_file)
CAM3_RCP45_LR_EnergyB_Annual_GA_file.close()

CAM3_RCP45_LR_EnergyB_SON_GA_file = open("CAM3_RCP45_LR_EnergyB_SON_GA.pickle","wb")
pickle.dump(CAM3_RCP45_LR_EnergyB_SON_GA,CAM3_RCP45_LR_EnergyB_SON_GA_file)
CAM3_RCP45_LR_EnergyB_SON_GA_file.close()

CAM3_RCP45_LR_EnergyB_JJA_GA_file = open("CAM3_RCP45_LR_EnergyB_JJA_GA.pickle","wb")
pickle.dump(CAM3_RCP45_LR_EnergyB_JJA_GA,CAM3_RCP45_LR_EnergyB_JJA_GA_file)
CAM3_RCP45_LR_EnergyB_JJA_GA_file.close()

CAM3_RCP45_LR_EnergyB_MAM_GA_file = open("CAM3_RCP45_LR_EnergyB_MAM_GA.pickle","wb")
pickle.dump(CAM3_RCP45_LR_EnergyB_MAM_GA,CAM3_RCP45_LR_EnergyB_MAM_GA_file)
CAM3_RCP45_LR_EnergyB_MAM_GA_file.close()

CAM3_RCP45_LR_EnergyB_DJF_GA_file = open("CAM3_RCP45_LR_EnergyB_DJF_GA.pickle","wb")
pickle.dump(CAM3_RCP45_LR_EnergyB_DJF_GA,CAM3_RCP45_LR_EnergyB_DJF_GA_file)
CAM3_RCP45_LR_EnergyB_DJF_GA_file.close()

CAM3_RCP45_LR_Feedback_Annual_GA_file = open("CAM3_RCP45_LR_Feedback_GA.pickle","wb")
pickle.dump(CAM3_RCP45_LR_Feedback_Annual_GA,CAM3_RCP45_LR_Feedback_Annual_GA_file)
CAM3_RCP45_LR_Feedback_Annual_GA_file.close()

CAM3_RCP45_LR_Feedback_SON_GA_file = open("CAM3_RCP45_LR_Feedback_SON_GA.pickle","wb")
pickle.dump(CAM3_RCP45_LR_Feedback_SON_GA,CAM3_RCP45_LR_Feedback_SON_GA_file)
CAM3_RCP45_LR_Feedback_SON_GA_file.close()

CAM3_RCP45_LR_Feedback_JJA_GA_file = open("CAM3_RCP45_LR_Feedback_JJA_GA.pickle","wb")
pickle.dump(CAM3_RCP45_LR_Feedback_JJA_GA,CAM3_RCP45_LR_Feedback_JJA_GA_file)
CAM3_RCP45_LR_Feedback_JJA_GA_file.close()

CAM3_RCP45_LR_Feedback_MAM_GA_file = open("CAM3_RCP45_LR_Feedback_MAM_GA.pickle","wb")
pickle.dump(CAM3_RCP45_LR_Feedback_MAM_GA,CAM3_RCP45_LR_Feedback_MAM_GA_file)
CAM3_RCP45_LR_Feedback_MAM_GA_file.close()

CAM3_RCP45_LR_Feedback_DJF_GA_file = open("CAM3_RCP45_LR_Feedback_DJF_GA.pickle","wb")
pickle.dump(CAM3_RCP45_LR_Feedback_DJF_GA,CAM3_RCP45_LR_Feedback_DJF_GA_file)
CAM3_RCP45_LR_Feedback_DJF_GA_file.close()

In [46]:
CAM3_RCP45_PL_EnergyB_Annual_GA_file = open("CAM3_RCP45_PL_EnergyB_GA.pickle","wb")
pickle.dump(CAM3_RCP45_PL_EnergyB_Annual_GA,CAM3_RCP45_PL_EnergyB_Annual_GA_file)
CAM3_RCP45_PL_EnergyB_Annual_GA_file.close()

CAM3_RCP45_PL_EnergyB_SON_GA_file = open("CAM3_RCP45_PL_EnergyB_SON_GA.pickle","wb")
pickle.dump(CAM3_RCP45_PL_EnergyB_SON_GA,CAM3_RCP45_PL_EnergyB_SON_GA_file)
CAM3_RCP45_PL_EnergyB_SON_GA_file.close()

CAM3_RCP45_PL_EnergyB_JJA_GA_file = open("CAM3_RCP45_PL_EnergyB_JJA_GA.pickle","wb")
pickle.dump(CAM3_RCP45_PL_EnergyB_JJA_GA,CAM3_RCP45_PL_EnergyB_JJA_GA_file)
CAM3_RCP45_PL_EnergyB_JJA_GA_file.close()

CAM3_RCP45_PL_EnergyB_MAM_GA_file = open("CAM3_RCP45_PL_EnergyB_MAM_GA.pickle","wb")
pickle.dump(CAM3_RCP45_PL_EnergyB_MAM_GA,CAM3_RCP45_PL_EnergyB_MAM_GA_file)
CAM3_RCP45_PL_EnergyB_MAM_GA_file.close()

CAM3_RCP45_PL_EnergyB_DJF_GA_file = open("CAM3_RCP45_PL_EnergyB_DJF_GA.pickle","wb")
pickle.dump(CAM3_RCP45_PL_EnergyB_DJF_GA,CAM3_RCP45_PL_EnergyB_DJF_GA_file)
CAM3_RCP45_PL_EnergyB_DJF_GA_file.close()

CAM3_RCP45_PL_Feedback_Annual_GA_file = open("CAM3_RCP45_PL_Feedback_GA.pickle","wb")
pickle.dump(CAM3_RCP45_PL_Feedback_Annual_GA,CAM3_RCP45_PL_Feedback_Annual_GA_file)
CAM3_RCP45_PL_Feedback_Annual_GA_file.close()

CAM3_RCP45_PL_Feedback_SON_GA_file = open("CAM3_RCP45_PL_Feedback_SON_GA.pickle","wb")
pickle.dump(CAM3_RCP45_PL_Feedback_SON_GA,CAM3_RCP45_PL_Feedback_SON_GA_file)
CAM3_RCP45_PL_Feedback_SON_GA_file.close()

CAM3_RCP45_PL_Feedback_JJA_GA_file = open("CAM3_RCP45_PL_Feedback_JJA_GA.pickle","wb")
pickle.dump(CAM3_RCP45_PL_Feedback_JJA_GA,CAM3_RCP45_PL_Feedback_JJA_GA_file)
CAM3_RCP45_PL_Feedback_JJA_GA_file.close()

CAM3_RCP45_PL_Feedback_MAM_GA_file = open("CAM3_RCP45_PL_Feedback_MAM_GA.pickle","wb")
pickle.dump(CAM3_RCP45_PL_Feedback_MAM_GA,CAM3_RCP45_PL_Feedback_MAM_GA_file)
CAM3_RCP45_PL_Feedback_MAM_GA_file.close()

CAM3_RCP45_PL_Feedback_DJF_GA_file = open("CAM3_RCP45_PL_Feedback_DJF_GA.pickle","wb")
pickle.dump(CAM3_RCP45_PL_Feedback_DJF_GA,CAM3_RCP45_PL_Feedback_DJF_GA_file)
CAM3_RCP45_PL_Feedback_DJF_GA_file.close()